In [120]:
import sys
notebook_path = sys.path.append(r'e:\Data science\Titanic dataset\notebooks')

# Importing main libraries
from auto_imports import *
from sklearn.preprocessing import TargetEncoder , StandardScaler
from sklearn.linear_model import LogisticRegression

In [121]:
#Loading dataset
logistic_df = pd.read_csv('E:\Data science\Titanic dataset\data\Processed data\Data Analysis\processed_data.csv')
logistic_df.head() 

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ind,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
0,1,0.0,3,male,22.0,1,0,7.2500,S,train,23-25-27,C,Mr,Braund,A/5,0,0
1,2,1.0,1,female,38.0,1,0,71.2833,C,train,85,C,Mrs,Cumings,PC,0,0
2,3,1.0,3,female,26.0,0,0,7.9250,S,train,23-25-27,C,Miss,Heikkinen,STON/O2,0,0
3,4,1.0,1,female,35.0,1,0,53.1000,S,train,123,C,Mrs,Futrelle,Non-special,0,0
4,5,0.0,3,male,35.0,0,0,8.0500,S,train,23-25-27,C,Mr,Allen,Non-special,0,0


In [122]:
logistic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          1280 non-null   int64  
 1   Survived             871 non-null    float64
 2   Pclass               1280 non-null   int64  
 3   Sex                  1280 non-null   object 
 4   Age                  1280 non-null   float64
 5   SibSp                1280 non-null   int64  
 6   Parch                1280 non-null   int64  
 7   Fare                 1280 non-null   float64
 8   Embarked             1280 non-null   object 
 9   ind                  1280 non-null   object 
 10  cell_number          1280 non-null   object 
 11  Cabin_letter         1280 non-null   object 
 12  Title                1280 non-null   object 
 13  FamilyName           1280 non-null   object 
 14  Special Agent/route  1280 non-null   object 
 15  TicketBatch          1280 non-null   i

In [123]:
# Splitting the dataset to train_logistic_df and predict_df
train_logistic_df = logistic_df.query('`ind` == "train"').drop(['ind' , 'PassengerId'] , axis=1)
predict_df = logistic_df.query('`ind` == "test"').drop(['ind' , 'Survived'] , axis=1)

In [124]:
train_logistic_df.head() # take a look on training data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
0,0.0,3,male,22.0,1,0,7.2500,S,23-25-27,C,Mr,Braund,A/5,0,0
1,1.0,1,female,38.0,1,0,71.2833,C,85,C,Mrs,Cumings,PC,0,0
2,1.0,3,female,26.0,0,0,7.9250,S,23-25-27,C,Miss,Heikkinen,STON/O2,0,0
3,1.0,1,female,35.0,1,0,53.1000,S,123,C,Mrs,Futrelle,Non-special,0,0
4,0.0,3,male,35.0,0,0,8.0500,S,23-25-27,C,Mr,Allen,Non-special,0,0


In [125]:
for col in train_logistic_df.select_dtypes('O').columns: # show all unique values of all categorical data to see the best encoding for them
    print(f'Column: {col}\n \n',train_logistic_df[col].unique())

Column: Sex
 
 ['male' 'female']
Column: Embarked
 
 ['S' 'C' 'Q']
Column: cell_number
 
 ['23-25-27' '85' '123' '46' '6' '103' '56' '78' '33' '30' '52' '83' '73'
 '31' '5' '10-12' '26' '110' '58-60' '101' '69' '47' '86' '2' '19' '7'
 '49' '4' '32' '80' '36' '15' '93' '35' '87' '77' '67' '94' '125' '99'
 '118' '22-26' '106' '65' '54' '57-59-63-66' '34' '18' '124' '91' '40'
 '128' '37' '50' '82' '96-98' '44' '104' '111' '92' '38' '21' '12' '63'
 '14' '20' '79' '25' '95' '39' '22' '70' '16' '68' '10' '41' '9' '23' '48'
 '58' '126' '71' '51-53-55' '62-64' '24' '90' '45' '8' '121' '11' '3'
 '82-84' '17' '102' '28' '42' '148']
Column: Cabin_letter
 
 ['C' 'E' 'G' 'D' 'A' 'B' 'F' 'F-G' 'F-E']
Column: Title
 
 ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']
Column: FamilyName
 
 ['Braund' 'Cumings' 'Heikkinen' 'Futrelle' 'Allen' 'Moran' 'McCarthy'
 'Palsson' 'Johnson' 'Nasser' 'Sandstrom' 'Bonnell' 'Saundercock'
 'Andersso

I will make target encode for all columns except Sex and Embarked columns
> I will make One Hot Encoding on them

In [126]:
train_logistic_df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 871 entries, 0 to 870
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Survived             871 non-null    float64
 1   Pclass               871 non-null    int64  
 2   Sex                  871 non-null    object 
 3   Age                  871 non-null    float64
 4   SibSp                871 non-null    int64  
 5   Parch                871 non-null    int64  
 6   Fare                 871 non-null    float64
 7   Embarked             871 non-null    object 
 8   cell_number          871 non-null    object 
 9   Cabin_letter         871 non-null    object 
 10  Title                871 non-null    object 
 11  FamilyName           871 non-null    object 
 12  Special Agent/route  871 non-null    object 
 13  TicketBatch          871 non-null    int64  
 14  SharedTicket         871 non-null    int64  
dtypes: float64(3), int64(5), object(7)
memory usa

In [127]:
train_logistic_df['Survived'] = train_logistic_df['Survived'].astype(int) # convert survived data type 
"""(float64 --> int64)"""

'(float64 --> int64)'

In [128]:
train_logistic_df['Survived'].dtype

dtype('int64')

In [129]:
predict_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
871,892,3,male,34.5,0,0,7.8292,Q,23-25-27,C,Mr,Kelly,Non-special,0,0
872,893,3,female,47.0,1,0,7.0000,S,23-25-27,C,Mrs,Wilkes,Non-special,0,0
873,894,2,male,62.0,0,0,9.6875,Q,23-25-27,C,Mr,Myles,Non-special,0,0
874,895,3,male,27.0,0,0,8.6625,S,23-25-27,C,Mr,Wirz,Non-special,0,0
875,896,3,female,22.0,1,1,12.2875,S,23-25-27,C,Mrs,Hirvonen,Non-special,0,1


In [130]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 409 entries, 871 to 1279
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          409 non-null    int64  
 1   Pclass               409 non-null    int64  
 2   Sex                  409 non-null    object 
 3   Age                  409 non-null    float64
 4   SibSp                409 non-null    int64  
 5   Parch                409 non-null    int64  
 6   Fare                 409 non-null    float64
 7   Embarked             409 non-null    object 
 8   cell_number          409 non-null    object 
 9   Cabin_letter         409 non-null    object 
 10  Title                409 non-null    object 
 11  FamilyName           409 non-null    object 
 12  Special Agent/route  409 non-null    object 
 13  TicketBatch          409 non-null    int64  
 14  SharedTicket         409 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory 

In [131]:
x = train_logistic_df.drop('Survived' , axis=1)
y = train_logistic_df['Survived'] # our dependent variable

In [132]:
#Scalling numeric data
numeric_data = x.select_dtypes('float')

numeric_data

,Age,Fare
0,22.000000,7.2500
1,38.000000,71.2833
2,26.000000,7.9250
3,35.000000,53.1000
4,35.000000,8.0500
...,...,...
866,27.000000,13.0000
867,19.000000,30.0000
868,29.881138,23.4500
869,26.000000,30.0000


Ok in EDA I see there are many outlires in Fare column, so I will use Robust scaler on it
> Age column I will use standard scaler because of normal distributed

In [133]:
robust_scaler = RobustScaler()
standard_scaler = StandardScaler()

# Scaling fare column using robust column
x['Fare'] = robust_scaler.fit_transform(x['Fare'].values.reshape(-1,1))

#Scaling age column using standard scaler
x['Age'] = standard_scaler.fit_transform(x['Age'].values.reshape(-1,1))

#Applying scaling in predict data
predict_df['Fare'] = robust_scaler.transform(predict_df['Fare'].values.reshape(-1,1))
predict_df['Age'] = standard_scaler.transform(predict_df['Age'].values.reshape(-1,1))

In [134]:
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
0,3,male,-0.584430,1,0,-0.310164,S,23-25-27,C,Mr,Braund,A/5,0,0
1,1,female,0.643003,1,0,2.446672,C,85,C,Mrs,Cumings,PC,0,0
2,3,female,-0.277572,0,0,-0.281103,S,23-25-27,C,Miss,Heikkinen,STON/O2,0,0
3,1,female,0.412859,1,0,1.663824,S,123,C,Mrs,Futrelle,Non-special,0,0
4,3,male,0.412859,0,0,-0.275721,S,23-25-27,C,Mr,Allen,Non-special,0,0


In [135]:
# Using target encoder for categorical values becuase of high cardinality of categorical features
enc = TargetEncoder() 
categorical_columns = x.select_dtypes('O').drop(['Sex' , 'Embarked'] , axis =1).columns # Dropping Sex and Embarked column because it have only two values and one hot encoding will be the good choice
categorical_data_train = x.select_dtypes('O').drop(['Sex' , 'Embarked'] , axis =1)
categorical_data_predict = predict_df.select_dtypes('O').drop(['Sex' , 'Embarked'] , axis= 1)

In [136]:
# Fitting and trasforming
x[categorical_columns]= enc.fit_transform(x[categorical_columns] , y)
predict_df[categorical_columns] = enc.transform(categorical_data_predict[categorical_columns])

In [137]:
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket
0,3,male,-0.584430,1,0,-0.310164,S,0.293612,0.315974,0.152048,0.000000,0.141080,0,0
1,1,female,0.643003,1,0,2.446672,C,0.377874,0.322613,0.784956,0.377874,0.587442,0,0
2,3,female,-0.277572,0,0,-0.281103,S,0.300685,0.322613,0.689843,0.377874,0.000000,0,0
3,1,female,0.412859,1,0,1.663824,S,0.000000,0.314092,0.799068,0.000000,0.383257,0,0
4,3,male,0.412859,0,0,-0.275721,S,0.293494,0.320972,0.144944,1.000000,0.375715,0,0


In [138]:
# One hot encoding on Sex and Embarked column

x['Male'] = pd.get_dummies(x['Sex']  , 
                           dtype=int)['male'] # if value is 1 --> male , 0 --> female
x[['Embarked_C' , 'Embarked_S']] = pd.get_dummies(x['Embarked'] ,
                                                  dtype=int)[['C' , 'S']] # if two columns values is 0 --> Q

x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket,Male,Embarked_C,Embarked_S
0,3,male,-0.584430,1,0,-0.310164,S,0.293612,0.315974,0.152048,0.000000,0.141080,0,0,1,0,1
1,1,female,0.643003,1,0,2.446672,C,0.377874,0.322613,0.784956,0.377874,0.587442,0,0,0,1,0
2,3,female,-0.277572,0,0,-0.281103,S,0.300685,0.322613,0.689843,0.377874,0.000000,0,0,0,0,1
3,1,female,0.412859,1,0,1.663824,S,0.000000,0.314092,0.799068,0.000000,0.383257,0,0,0,0,1
4,3,male,0.412859,0,0,-0.275721,S,0.293494,0.320972,0.144944,1.000000,0.375715,0,0,1,0,1


In [139]:
predict_df['Male'] = pd.get_dummies(predict_df['Sex']  , 
                           dtype=int)['male'] # if value is 1 --> male , 0 --> female
predict_df[['Embarked_C' , 'Embarked_S']] = pd.get_dummies(predict_df['Embarked'] ,
                                                  dtype=int)[['C' , 'S']] # if two columns values is 0 --> Q

predict_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cell_number,Cabin_letter,Title,FamilyName,Special Agent/route,TicketBatch,SharedTicket,Male,Embarked_C,Embarked_S
871,892,3,male,0.374502,0,0,-0.285227,Q,0.296728,0.319367,0.147362,0.688103,0.378669,0,0,1,0,0
872,893,3,female,1.333434,1,0,-0.320927,S,0.296728,0.319367,0.784506,0.377727,0.378669,0,0,0,0,1
873,894,2,male,2.484152,0,0,-0.205221,Q,0.296728,0.319367,0.147362,0.377727,0.378669,0,0,1,0,0
874,895,3,male,-0.200857,0,0,-0.249351,S,0.296728,0.319367,0.147362,0.377727,0.378669,0,0,1,0,1
875,896,3,female,-0.584430,1,1,-0.093283,S,0.296728,0.319367,0.784506,1.000000,0.378669,0,1,0,0,1


In [140]:
# Dropping Sex and Embarked column

x.drop(['Sex' , 'Embarked'] , axis=1 , inplace=True)
predict_df.drop(['Sex' , 'Embarked'] , axis=1 , inplace=True)

In [141]:
# splitting x and y to x_train , x_test , y_train and y_test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Fitting the model
logit = LogisticRegression(random_state=42).fit(X_train, y_train)

In [142]:
(logit.score(X_test ,y_test),
 logit.score(X_train,y_train)) # Getting R2 and checking the overlapping

(0.7885714285714286, 0.8347701149425287)

$R^2 \approx 0.868$
> This a very goood percentage and indicates that model is very strong

In [143]:
# Saving the model to combine it with the oher models
import joblib

model_filename = r'E:\Data science\Titanic dataset\Models\LogisticRegression.pkl'

joblib.dump(logit , model_filename)

"""Model saved successfully"""

'Model saved successfully'

In [146]:
# Predicting predict data


predict_df['Survived_pred'] = logit.predict(predict_df.drop('PassengerId' , axis=1))